In [1]:
import os
import langchain
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.cache import InMemoryCache
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser, DatetimeOutputParser
from langchain.schema.output_parser import BaseLLMOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate

# Config
load_dotenv(".env")
api_key = os.environ.get("key")

In [2]:
langchain.llm_cache = InMemoryCache()
chat = ChatOpenAI(api_key=api_key, model='gpt-3.5-turbo')

In [3]:
class Quiz:
    def __init__(self) -> None:
        self.system_template = "You are AI assistant that specialize in creating multiple choice questions on {topic} and only return question and its options" 
        self.human_template = "{request_prompt}"
        self.datetime_parser = DatetimeOutputParser()
        
    def createQuiz(self, topic, user_prompt):
        """
        Info:
          A chatBot that create quiz based on topic given to it
        Args:
          topic --> Enter Topic
        Returns:
          Questions with options
        """
        system_prompt = SystemMessagePromptTemplate.from_template(self.system_template)
        human_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        chat_promt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
        prompt = chat_promt.format_prompt(topic=topic, request_prompt=user_prompt).to_messages()
        return chat(prompt).content
        
    def Answer(self, question):
        system_template = "Answer the question by returning correction option"
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = """Answer the user's questions:
        {question}"""
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
        prompt = chat_prompt.format_prompt(question=question).to_messages()
        return chat(prompt).content

In [4]:
bot = Quiz()
print(bot.createQuiz("machine learning", "Give on question on machine learning"))

Question: What is an artificial neural network?

Options:
a) A computational model inspired by the structure and functions of the human brain
b) A type of algorithm used for data clustering
c) A statistical method used for linear regression
d) A technique for feature extraction in computer vision


In [5]:
bot.Answer(bot.createQuiz("machine learning", "Give on question on machine learning"))

'a) A computational model inspired by the structure and functions of the human brain'